In [ ]:
import requests
import json

def searxng_search(query: str, searx_host: str, num_results: int = 10):
    """
    Faz uma busca no SearxNG e retorna JSON dos resultados.
    """
    url = f"{searx_host.rstrip('/')}/search"
    params = {
        "q": query,
        "format": "json",
        "pageno": 4,
        "results_on_new_tab": 0,
        # você pode adicionar outros parâmetros suportados (engines, language, etc.)
        "num_results": num_results,
        "language": "pt-br"
    }
    resp = requests.get(url, params=params, timeout=10)
    resp.raise_for_status()
    return resp.json()

# Exemplo de uso:
searx_host = "http://localhost:8888"  # ou sua instância SearxNG
resultado = searxng_search("Qual é o maior animal do mundo?", searx_host)

json_formatado = json.dumps(resultado, indent=4, ensure_ascii=False)
print(json_formatado)

{
    "query": "Qual é o maior animal do mundo?",
    "number_of_results": 0,
    "results": [
        {
            "url": "https://mundoeducacao.uol.com.br/biologia/baleia-azul.htm",
            "title": "Baleia-azul: habitat, características, curiosidades - Mundo Educação",
            "content": "A baleia-azul é o maior animal do mundo, sendo provavelmente o maior animal que já habitou o planeta. Esse incrível mamífero aquático pode atingir cerca de 30 metros de comprimento, o que corresponde a praticamente um prédio de 10 andares. Além disso, pode pesar mais de 150 toneladas.",
            "thumbnail": "",
            "engine": "google",
            "template": "default.html",
            "parsed_url": [
                "https",
                "mundoeducacao.uol.com.br",
                "/biologia/baleia-azul.htm",
                "",
                "",
                ""
            ],
            "img_src": "",
            "priority": "",
            "engines": [
             